<a href="https://colab.research.google.com/github/lukaszplust/Projects/blob/main/B_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
class Record:
    def __init__(self, key, data):
        self.key = key
        self.data = data

    def __repr__(self):
        return f"Record({self.key}, {self.data})"

    def to_bytes(self):
        key_bytes = self.key.to_bytes(4, byteorder='big')
        data_bytes = self.data.encode()
        length_bytes = len(data_bytes).to_bytes(4, byteorder='big')
        return key_bytes + length_bytes + data_bytes

    @staticmethod
    def from_bytes(byte_data):
        key = int.from_bytes(byte_data[:4], byteorder='big')
        length = int.from_bytes(byte_data[4:8], byteorder='big')
        data = byte_data[8:8+length].decode()
        return Record(key, data)

In [16]:
class ReadBuffer:
    def __init__(self, path, buffer_size=4):
        self.path = path
        self.buffer_size = buffer_size
        self.buffer = []
        self.file = open(self.path, "rb")
        self.position = 0
        self.disk_reads_count = 0

    def read_next(self):
        if self.position >= len(self.buffer):
            self._fill_buffer()
        if not self.buffer:
            return None
        record = self.buffer[self.position]
        self.position += 1
        return record

    def _fill_buffer(self):
        self.buffer = []
        self.position = 0
        for _ in range(self.buffer_size):
            bytes_record = self.file.read(8)
            if not bytes_record:
                break
            length = int.from_bytes(bytes_record[4:8], byteorder='big')
            data = self.file.read(length)
            self.buffer.append(Record.from_bytes(bytes_record + data))
        self.disk_reads_count += 1

    def close(self):
        self.file.close()

In [17]:
class WriteBuffer:
    def __init__(self, path, buffer_size=4):
        self.path = path
        self.buffer_size = buffer_size
        self.buffer = []
        self.disk_writes_count = 0

    def write(self, record):
        self.buffer.append(record)
        if len(self.buffer) >= self.buffer_size:
            self.flush()

    def flush(self):
        with open(self.path, "ab") as file:
            for record in self.buffer:
                file.write(record.to_bytes())
        self.disk_writes_count += 1
        self.buffer = []

    def close(self):
        if self.buffer:
            self.flush()

In [18]:
class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t  # Minimalny stopień (t)
        self.leaf = leaf  # Czy jest liściem
        self.keys = []  # Lista kluczy
        self.children = []  # Lista dzieci (węzłów)

    def __repr__(self):
        return f'BTreeNode(keys={self.keys}, leaf={self.leaf})'

    def find_key(self, k):
        idx = 0
        while idx < len(self.keys) and self.keys[idx] < k:
            idx += 1
        return idx

    def insert_non_full(self, k):
        i = len(self.keys) - 1
        if self.leaf:
            self.keys.append(None)
            while i >= 0 and self.keys[i] > k:
                self.keys[i + 1] = self.keys[i]
                i -= 1
            self.keys[i + 1] = k
        else:
            while i >= 0 and self.keys[i] > k:
                i -= 1
            i += 1
            if len(self.children[i].keys) == 2 * self.t - 1:
                self.split_child(i)
                if self.keys[i] < k:
                    i += 1
            self.children[i].insert_non_full(k)

    def split_child(self, i):
        t = self.t
        y = self.children[i]
        z = BTreeNode(t, y.leaf)
        self.children.insert(i + 1, z)
        self.keys.insert(i, y.keys[t - 1])
        z.keys = y.keys[t:(2 * t) - 1]
        y.keys = y.keys[0:t - 1]
        if not y.leaf:
            z.children = y.children[t:(2 * t)]
            y.children = y.children[0:t - 1]

In [20]:
class BTree:
    def __init__(self, t):
        self.root = BTreeNode(t, leaf=True)
        self.t = t

    def traverse(self):
        self._traverse(self.root)
        print()

    def _traverse(self, node):
        for i in range(len(node.keys)):
            if not node.leaf:
                self._traverse(node.children[i])
            print(node.keys[i], end=' ')
        if not node.leaf:
            self._traverse(node.children[len(node.keys)])

    # Sprawdza, czy klucz istnieje w drzewie. Jeśli klucz jest obecny,
    # metoda zwraca go oraz dane powiązane (w przypadku węzłów wewnętrznych,
    # odnosi się do dzieci węzłów), w przeciwnym razie zwraca None
    def search(self, k):
        return self._search(self.root, k)

    def _search(self, node, k):
        i = node.find_key(k)

        if i < len(node.keys) and node.keys[i] == k:
            return (node.keys[i], node.children[i] if not node.leaf else None)

        if node.leaf:
            return None

        return self._search(node.children[i], k)

    def insert(self, k):
        if self.search(k) is not None:
            print(f'Already exists: {k}')
            return

        root = self.root
        if len(root.keys) == (2 * self.t) - 1:
            temp = BTreeNode(self.t)
            self.root = temp
            temp.children.insert(0, root)
            self._split_child(temp, 0)
            self._insert_non_full(temp, k)
        else:
            self._insert_non_full(root, k)

    def _insert_non_full(self, node, k):
        node.insert_non_full(k)

    def _split_child(self, node, i):
        node.split_child(i)

if __name__ == "__main__":
    btree = BTree(2)
    btree.insert(10)
    btree.insert(20)
    btree.insert(5)
    btree.insert(6)
    btree.insert(12)
    btree.insert(30)
    btree.insert(7)
    btree.insert(17)

    print("Traversal of tree constructed:")
    btree.traverse()

    k = 6
    result = btree.search(k)
    if result:
        print(f"\nFound: {result}")
    else:
        print(f"\nNot found: {k}")

    k = 15
    result = btree.search(k)
    if result:
        print(f"\nFound: {result}")
    else:
        print(f"\nNot found: {k}")


Traversal of tree constructed:
5 6 7 10 12 17 20 30 

Found: (6, None)

Not found: 15
